<a href="https://colab.research.google.com/github/KontainPluton/TPs_Artificial_Neural_Network/blob/main/%5BTP_3%5D_%5BCorrection%5D_Text_classification_with_Transformers_and_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 21.2 MB/s 
     |████████████████████████████████| 6.6 MB 57.4 MB/s 
     |████████████████████████████████| 163 kB 63.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 431 kB 29.8 MB/s 
     |████████████████████████████████| 212 kB 67.6 MB/s 
     |████████████████████████████████| 115 kB 65.7 MB/s 
     |████████████████████████████████| 127 kB 73.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
from datasets import list_datasets, load_dataset

# Load a dataset and print the first examples in the training set
yelp_dataset = load_dataset('yelp_polarity')

print(yelp_dataset)

training_set = yelp_dataset['train']
test_set = yelp_dataset['test']

print("Training set: {} ".format(len(training_set)))
print("Test set    : {} ".format(len(test_set)))

print("Entry train: {}".format(training_set[0]))

/usr/local/lib/python3.7/dist-packages/huggingface_hub/utils/_deprecation.py:97: FutureWarning: Deprecated argument(s) used in 'dataset_info': token. Will not be supported from version '0.12'.
  warnings.warn(message, FutureWarning)


Generating train split:   0%|          | 0/560000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/38000 [00:00<?, ? examples/s]

Dataset yelp_polarity downloaded and prepared to /root/.cache/huggingface/datasets/yelp_polarity/plain_text/1.0.0/14f90415c754f47cf9087eadac25823a395fef4400c7903c5897f55cfaaa6f61. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 560000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 38000
    })
})
Training set: 560000 
Test set    : 38000 
Entry train: {'text': "Unfortunately, the frustration of being Dr. Goldberg's patient is a repeat of the experience I've had with so many other doctors in NYC -- good doctor, terrible staff.  It seems that his staff simply never answers the phone.  It usually takes 2 hours of repeated calling to get an answer.  Who has time for that or wants to deal with it?  I have run into this problem with many other doctors and I just don't get it.  You have office workers, you have patients with medical needs, why isn't anyone answering the phone?  It's incomprehensible and not work the aggravation.  It's with regret that I feel that I have to give Dr. Goldberg 2 stars.", 'label': 0}


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# List of pretrained models: https://huggingface.co/models?filter=text-classification
tokenizer = AutoTokenizer.from_pretrained("distilroberta-base")
model = AutoModelForSequenceClassification.from_pretrained("distilroberta-base")

# We can also use a model fine tuned on Yelp
#tokenizer = AutoTokenizer.from_pretrained("textattack/albert-base-v2-yelp-polarity")
#model = AutoModelForSequenceClassification.from_pretrained("textattack/albert-base-v2-yelp-polarity")

print(tokenizer)
print(model)

Downloading:   0%|          | 0.00/480 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/331M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.dense.bias', 

PreTrainedTokenizerFast(name_or_path='distilroberta-base', vocab_size=50265, model_max_len=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False)})
RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_

As you can see `(classifier): Linear(in_features=768, out_features=2, bias=True)`
The classification head is just a linear model.

In [ ]:
from transformers import TextClassificationPipeline

albert_classifier = TextClassificationPipeline(model=model, tokenizer=tokenizer, framework='pt') # pt for PyTorch, device=-1 cpu, >= 0 -> GPU id

***

**Exercice:** Develop a simple function that can be used to evaluate your `TextClassificationPipeline` on the test set. This function will later be used to monitor the performance of our model.
* name your function `evaluate_yelp`
* enable to only perform evaluation only on `k` first entries:  `n_pred` parameter
* The function should compute the accuracy considering the tested set of entries

Finally use your function to evaluate your model on the first 100 entries of the test set.

From the documentation you can see 
https://huggingface.co/transformers/model_doc/albert.html : `max_position_embeddings=512`
Be sure you don't defined as input longer sequences.

Documentation related to Tokenizert preprocessing: 
https://huggingface.co/transformers/preprocessing.html

In [ ]:
def evaluate_yelp(yelp_classifier, nb_pred=100, verbose=False):
  
  print("running...")

  nb_pred_correct = 0

  if nb_pred <=0 or nb_pred > len(yelp_dataset['test']):
    nb_pred = len(yelp_dataset['test'])

  for id, entry in enumerate(yelp_dataset['test']):

    label, text = entry['label'], entry['text']

    # print(yelp_pipeline(text)) # get used to check the values generated by your pipeline
    # outpout is a list of dic storing for each provided input a 'label' in {'LABEL_0', 'LABEL_1'} and a 'score'

    # Some modifications have therefore to be made
    # Flags used for dataset labels and model outputs must match
    # model flags are here LABEL_0 (0) and LABEL_0 (1) 
    # dataset labels are 0/1
    label = "LABEL_0" if label == 0 else "LABEL_1" 

    # The model has been trained for texts with max len of 512
    # This could also be managed from the tokenizer
    text  = text[:512] if len(text) > 512 else text

    prediction = yelp_classifier(text)[0]['label']
    score = yelp_classifier(text)[0]['score']
    valid_pred = label == prediction
    
    if verbose or id % 500 == 0: 
      print("{}/{} expected {} pred: {}, with score: {:.3f} correct: {}: {}"
      .format(id+1,nb_pred, label, prediction, score, label == prediction, text))
    
    if valid_pred:
      nb_pred_correct += 1

    if id+1 == nb_pred:
      break

  acc = 100 * nb_pred_correct / nb_pred
  print("Accuracy ({} entries): {}%".format(nb_pred, acc))
  return acc  

In [ ]:
evaluate_yelp(albert_classifier, nb_pred=100, verbose=True)

running...
1/100 expected LABEL_1 pred: LABEL_0, with score: 0.532 correct: False: Contrary to other reviews, I have zero complaints about the service or the prices. I have been getting tire service here for the past 5 years now, and compared to my experience with places like Pep Boys, these guys are experienced and know what they're doing. \nAlso, this is one place that I do not feel like I am being taken advantage of, just because of my gender. Other auto mechanics have been notorious for capitalizing on my ignorance of cars, and have sucked my bank account dry. But here, my service and
2/100 expected LABEL_0 pred: LABEL_0, with score: 0.534 correct: True: Last summer I had an appointment to get new tires and had to wait a super long time. I also went in this week for them to fix a minor problem with a tire they put on. They \""fixed\"" it for free, and the very next morning I had the same issue. I called to complain, and the \""manager\"" didn't even apologize!!! So frustrated. Neve

48.0

**Exercice**

Compare the performance obtained using another model: `VictorSanh/roberta-base-finetuned-yelp-polarity`.
Make sure you're using a GPU now

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TextClassificationPipeline

def load_model_tok(model_flag):
  model = AutoModelForSequenceClassification.from_pretrained(model_flag)
  tokenizer = AutoTokenizer.from_pretrained(model_flag)
  return model, tokenizer 

def eval(model_flags):
  device = 0 # -1 cpu, val >= 0 -> GPU id

  for flag in model_flags:
    model, tok = load_model_tok(flag)
    classifier = TextClassificationPipeline(model=model, tokenizer=tok, framework='pt', device=device)
    acc = evaluate_yelp(classifier, nb_pred=3000, verbose=False)
    print("{} \t {:.2f}".format(flag, acc))


model_flags = [
               'textattack/albert-base-v2-yelp-polarity',
               'VictorSanh/roberta-base-finetuned-yelp-polarity',
               'distilbert-base-uncased'
]

eval(flags)

NameError: ignored

* textattack/albert-base-v2-yelp-polarity 	 94.90
* VictorSanh/roberta-base-finetuned-yelp-polarity 	 96.27
* distilbert-base-uncased 	 54.10

**Exercice**

We will now apply fine-tuning on `distilbert-base-uncased`.

To do so you'll use the `Trainer` provided by Hugging Face : https://huggingface.co/transformers/main_classes/trainer.html

Note that you'll have to construct a `torch.utils.data.Dataset` for the Trainer.

Follow the following steps: 
- From the original Dataset loaded from Hugging face `datasets`, get the strings corresponding to the texts and associated labels into two lists. Do not use the whole dataset, 10000 entries for training and 3000 for validation (using the Trainer) will be enough. Debug with less entries first. 
- Tokenize the strings using directly the tokenizer, setting `truncation=True, padding=True`. This step will take a bit of time. 
- Generate the dataset using those structures and the code provided below, see `YelpDataset`.
- Train the model using the trainer. Configure Trainer using the provided configuration (see below). Be careful of the size of the Dataset you use and the training configuration. We just provide an illustration here. Training time may vary depending on the availability of Colab resources.  

Training the model you'll fine-tune it on this specific dataset. 
Next, evaluate the fine-tuned model again. 


In [ ]:
import torch
from datasets import list_datasets, load_dataset
from transformers import Trainer, TrainingArguments
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification

# We load DistilBert and associated tokenizer
# https://huggingface.co/distilbert-base-uncased
tokenizer_distilbert, model_distilbert_ft = load_tok_model("distilbert-base-uncased")

# Load the dataset
yelp_dataset = load_dataset('yelp_polarity')

print(yelp_dataset)

training_set = yelp_dataset['train']
test_set = yelp_dataset['test']

print("Training set: {} ".format(len(training_set)))
print("Test set    : {} ".format(len(test_set)))

# See what's inside a dataset
print(type(training_set))
print(training_set.features)

size_training_set = 10000
size_validation_set = 3000

training_set_text = training_set['text'][:size_training_set]
training_set_label = training_set['label'][:size_training_set]

validation_set_text = training_set['text'][size_training_set:size_training_set + size_validation_set]
validation_set_label = training_set['label'][size_training_set:size_training_set + size_validation_set]


print("Tokenizing...")
train_encodings = tokenizer_distilbert(training_set_text, truncation=True, padding=True)
val_encodings  = tokenizer_distilbert(validation_set_text, truncation=True, padding=True)


class YelpDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()} # tok generated input_ids, token_type_ids, attention_mask...
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = YelpDataset(train_encodings, training_set_label)
val_dataset = YelpDataset(val_encodings, validation_set_label)


training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=1,              # total number of training epochs: 3
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=50,
)


trainer = Trainer(
    model=model_distilbert_ft,        # the instantiated 🤗 Transformers model to be trained
    args=training_args,               # training arguments, defined above
    train_dataset=train_dataset,      # training dataset
    eval_dataset=val_dataset          # evaluation dataset
)

trainer.train()

NameError: ignored

In [ ]:
classifier_ft = TextClassificationPipeline(model=model_distilbert_ft, tokenizer=tokenizer_distilbert, framework='pt', device=device)
acc = evaluate_yelp(classifier_ft, nb_pred=3000, verbose=False)
print("{} \t {:.2f}".format(flag, acc))

NameError: ignored

* Distilbert-base-uncased Not Fine-tuned 54.10
* Distilbert-base-uncased Fine-tuned	   90.67


**Exercice 4**

Implement the training loop by yourself.

In [ ]:
from torch.utils.data import DataLoader
from transformers import DistilBertForSequenceClassification, AdamW

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')
model.to(device)
model.train()

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

optim = AdamW(model.parameters(), lr=5e-5)
nb_epochs = 1
log_frequency = 10


total_steps_epoch = len(train_loader)

for epoch in range(nb_epochs):

    step = 0

    for batch in train_loader:
        optim.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device) # used to take care of padding
        labels = batch['labels'].to(device)

        # Have a look to the corresponding structures
        #print(input_ids[0])
        #print(attention_mask[0])
        #print(batch['labels'][0].item())
        #break
        
       

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels, return_dict=True) 
        # Setting return_dict=True HuggingFace DistilBertForSequenceClassification return the loss, the logits...   
        #print(outputs)
        #break

        loss = outputs.loss
        loss.backward()
        optim.step()

        if step % log_frequency == 0:
          print("Epoch {}/{}   step {}/{}  loss: {}".format(epoch+1,nb_epochs, step+1, total_steps_epoch, loss))

        step += 1

model.eval()

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.bias', 'pre_classi

NameError: ignored

**Exercice 5**

Exercice: use the model to make predictions on new data

Just call the model :)